<p style="font-weight:bold;"> <span style="font-size: 36px"> Specifications Setup </span> </p>

In [ ]:
#!import "../Import/ImportScopeCalculation"
#!import "../Utils/TestHelper"

In [ ]:
public interface Universe : IMutableScopeWithStorage<ImportStorage>{}

public async Task<Universe> StartCalculatorAsync(string reportingNode, int year, int month, string economicBasis, Periodicity period, string format) 
{
    
    var importArgs = new ImportArgs(reportingNode, year, month, period, null, format);
    Workspace.Reset();
    Workspace.Initialize(x => x.FromSource(DataSource)
                         .DisableInitialization<RawVariable>()
                         .DisableInitialization<IfrsVariable>());

    var partition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == importArgs.ReportingNode &&
                                                                                                   p.Year == importArgs.Year &&
                                                                                                   p.Month == importArgs.Month &&
                                                                                                   p.Scenario == importArgs.Scenario
                                                                                            ).ToArrayAsync()).Single().Id;
    await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(partition);

    //Set Workspace as when the parsing is completed.
    if(importArgs.ImportFormat == ImportFormats.Actual)
    { 
        await Workspace.UpdateAsync<IfrsVariable>(await DataSource.Query<IfrsVariable>()
                                                      .Where(x => x.Partition == partition && 
                                                                  new string[]{EstimateTypes.A, EstimateTypes.AA, EstimateTypes.OA}.Contains(x.EstimateType) &&  
                                                                  new string[]{AocTypes.CF, AocTypes.WO}.Contains(x.AocType))
                                                      .ToArrayAsync());
    }
    else if(importArgs.ImportFormat == ImportFormats.Opening)
    { 
        await Workspace.UpdateAsync<IfrsVariable>(await DataSource.Query<IfrsVariable>()
                                                  .Where(x => x.Partition == partition && 
                                                                      new string[]{ EstimateTypes.C, EstimateTypes.L, EstimateTypes.LR, EstimateTypes.AA, EstimateTypes.OA, EstimateTypes.DA}.Contains(x.EstimateType) &&  
                                                                  x.AocType == AocTypes.BOP && x.Novelty == Novelties.I)
                                                      .ToArrayAsync());
    }
    else
    {
        await Workspace.UpdateAsync<RawVariable>(await DataSource.Query<RawVariable>().Where(x => x.Partition == partition).ToArrayAsync());
    }
    var storage = new ImportStorage(importArgs, DataSource, Workspace);
    await storage.InitializeAsync();
    return Scopes.ForStorage(storage).WithContext(economicBasis).ToScope<Universe>();
}

public static ImportIdentity GetIdentity (this Universe universe, string dataNode, string aocType, string novelty) 
    => universe.GetScope<GetIdentities>(dataNode).Identities.Single( x => x.AocType == aocType && x.Novelty == novelty );

static T GetValidElement<T>(this IList<T> array, int index) => GetElementOrDefault(array, index);

static T GetValidElement<T>(this IEnumerable<T> array, int index) => GetElementOrDefault(array.ToArray(), index);

static void ForEach<T>(this IEnumerable<T> self, Action<T> action) => self.ToList().ForEach(action);

public static IfrsVariable FromPvToIfrsVariable(this PresentValue scope)
    => new IfrsVariable{ EconomicBasis = scope.EconomicBasis, 
                         EstimateType = scope.Identity.CalculationType, 
                         DataNode = scope.Identity.Id.DataNode, 
                         AocType = scope.Identity.Id.AocType, 
                         Novelty = scope.Identity.Id.Novelty, 
                         AccidentYear = scope.Identity.AccidentYear,
                         AmountType = scope.Identity.AmountType,
                         Value = scope.Value,
                         Partition = scope.GetStorage().TargetPartition };

public static IfrsVariable FromCsmToIfrsVariable(this Csm scope)
    => new IfrsVariable{ EstimateType = scope.EstimateType, 
                         DataNode = scope.Identity.DataNode, 
                         AocType = scope.Identity.AocType, 
                         Novelty = scope.Identity.Novelty, 
                         Value = scope.Value,
                         Partition = scope.GetStorage().TargetPartition };

public static IfrsVariable FromLcToIfrsVariable(this Lc scope)
    => new IfrsVariable{ EstimateType = scope.EstimateType, 
                         DataNode = scope.Identity.DataNode, 
                         AocType = scope.Identity.AocType, 
                         Novelty = scope.Identity.Novelty, 
                         Value = scope.Value,
                         Partition = scope.GetStorage().TargetPartition };

public static IfrsVariable FromLoReCoToIfrsVariable(this LoReCo scope)
    => new IfrsVariable{ EstimateType = scope.EstimateType, 
                         DataNode = scope.Identity.DataNode, 
                         AocType = scope.Identity.AocType, 
                         Novelty = scope.Identity.Novelty, 
                         Value = scope.Value,
                         Partition = scope.GetStorage().TargetPartition };